# LAB | Hyperparameter Tuning

**Load the data**

Finally step in order to maximize the performance on your Spaceship Titanic model.

The data can be found here:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv

Metadata

https://github.com/data-bootcamp-v4/data/blob/main/spaceship_titanic.md

So far we've been training and evaluating models with default values for hyperparameters.

Today we will perform the same feature engineering as before, and then compare the best working models you got so far, but now fine tuning it's hyperparameters.

In [134]:
from sklearn.datasets import  fetch_california_housing
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer, KNNImputer

from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error,root_mean_squared_error  

In [135]:
df = pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/spaceship_titanic.csv")
df.head()

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True


In [136]:

df["Age"] = df["Age"].astype(float).astype("Int64")
df["HomePlanet"]=df["HomePlanet"].replace("Europa","Earth")
df["HomePlanet"].unique()
df = df.drop(columns=["Name","PassengerId"], errors="ignore")

#for numbers, takes the data that is most similar in other rows and it fill ins
imputer = KNNImputer(n_neighbors=5)
df_num = imputer.fit_transform(df[["Age", "RoomService", "Spa","ShoppingMall","FoodCourt","VRDeck"]])
df[["Age", "RoomService", "Spa","ShoppingMall","FoodCourt","VRDeck"]] = df_num
df[["Age", "RoomService", "Spa","ShoppingMall","FoodCourt","VRDeck"]].isna().sum()
#for booleans we use Simple Imputer

bools=["VIP","CryoSleep"]
imputer_bool=SimpleImputer(strategy="most_frequent")
df[bools]=imputer_bool.fit_transform(df[bools])
df[["VIP","CryoSleep"]].isna().sum()
cat_cols = ["HomePlanet", "Cabin", "Destination"]

imputer_cat = SimpleImputer(strategy="most_frequent")
df[cat_cols] = imputer_cat.fit_transform(df[cat_cols])
df["Cabin"] = df["Cabin"].str[0]
cat_cols = ["HomePlanet", "Cabin", "Destination", "VIP", "CryoSleep"]
df = pd.get_dummies(df, columns=cat_cols, drop_first=True)

df.head()

y=df["Transported"].astype(int)
X= df.drop(columns=["Transported"])
y

0       0
1       1
2       0
3       0
4       1
       ..
8688    0
8689    0
8690    1
8691    0
8692    1
Name: Transported, Length: 8693, dtype: int64

Now perform the same as before:
- Feature Scaling
- Feature Selection


In [137]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=42,stratify=y)

- Now let's use the best model we got so far in order to see how it can improve when we fine tune it's hyperparameters.

In [138]:
normalizer = MinMaxScaler()
normalizer.fit(X_train)
X_train_norm = normalizer.transform(X_train)
X_test_norm  = normalizer.transform(X_test)
X_train_norm = pd.DataFrame(X_train_norm, columns=X_train.columns)
X_test_norm  = pd.DataFrame(X_test_norm,  columns=X_test.columns)

- Evaluate your model

In [139]:
base_tree=DecisionTreeClassifier(random_state=42)
ada_clf = AdaBoostClassifier(
    estimator=base_tree,  
    random_state=42
)

**Grid/Random Search**

For this lab we will use Grid Search.

- Define hyperparameters to fine tune.

In [140]:
grid = {"n_estimators": [50, 100, 200,500],
        "estimator__max_leaf_nodes": [250, 500, 1000, None],
        "estimator__max_depth":[10,30,50]}


model = GridSearchCV(
    estimator=ada_clf,
    param_grid=grid,
    cv=5,
    scoring="r2",   
    n_jobs=-1,
    verbose=2,
    refit=True
)


model.fit(X_train_norm, y_train)

print("Mejores parámetros encontrados:")
print(model.best_params_)
print("\nMejor R2 medio en CV:")
print(model.best_score_)
best_model = model.best_estimator_

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Mejores parámetros encontrados:
{'estimator__max_depth': 10, 'estimator__max_leaf_nodes': None, 'n_estimators': 50}

Mejor R2 medio en CV:
0.12564003568579907


- Run Grid Search

In [142]:

model = RandomizedSearchCV(estimator = ada_clf, param_distributions = grid, n_iter = 10, cv = 5, n_jobs = -1)
model.fit(X_train_norm,y_train)
model.best_params_

{'n_estimators': 200,
 'estimator__max_leaf_nodes': 250,
 'estimator__max_depth': 30}

In [143]:
best_model = model.best_estimator_

- Evaluate your model

In [144]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np

pred = best_model.predict(X_test_norm)

print("MAE :", mean_absolute_error(y_test, pred))

mse  = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

print("R2  :", r2_score(y_test, pred))

MAE : 0.20701552616446234
RMSE: 0.45498958907260983
R2  : 0.17189161732725922
